In [3]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import MinMaxScaler
import matplotlib.pyplot as plt
import seaborn as sns

# ======================
# 1. DATA LOADING WITH SAFETY CHECKS
# ======================

def load_data_safely():
    """Load all USDA datasets with validation checks"""
    try:
        # Load all 5 datasets with essential columns
        food = pd.read_csv('food.csv', usecols=['fdc_id', 'description', 'food_category_id'])
        nutrient = pd.read_csv('nutrient.csv', usecols=['id', 'name', 'unit_name'])
        food_nutrient = pd.read_csv('food_nutrient.csv', 
                                  usecols=['fdc_id', 'nutrient_id', 'amount'])
        category = pd.read_csv('food_category.csv', usecols=['id', 'description'])
        measure_unit = pd.read_csv('measure_unit.csv', usecols=['id', 'name', 'abbreviation'])
        
        # Validate critical columns
        for df, cols in zip([food, nutrient, food_nutrient, category, measure_unit],
                           [['fdc_id'], ['id'], ['amount'], ['id'], ['abbreviation']]):
            assert all(c in df.columns for c in cols), f"Missing columns in {df}"
        
        return food, nutrient, food_nutrient, category, measure_unit
    
    except Exception as e:
        print(f"🚨 Data loading failed: {str(e)}")
        raise

In [4]:
# ======================
# 2. POSTPARTUM REQUIREMENTS (COMPREHENSIVE)
# ======================

POSTPARTUM_RDA = {
    # Macronutrients
    'Protein': {'target': 71, 'unit': 'g', 'critical': True},
    'Total lipid (fat)': {'target': 70, 'unit': 'g', 'critical': False},
    
    # Micronutrients
    'Iron, Fe': {'target': 9, 'unit': 'mg', 'critical': True},
    'Calcium, Ca': {'target': 1000, 'unit': 'mg', 'critical': True},
    'Vitamin B-12': {'target': 2.8, 'unit': 'µg', 'critical': True},
    'Folate, total': {'target': 600, 'unit': 'µg', 'critical': True},
    'Vitamin D (D2 + D3)': {'target': 15, 'unit': 'µg', 'critical': True},
    'Choline, total': {'target': 550, 'unit': 'mg', 'critical': False},
    
    # Breastfeeding adjustments
    'Energy': {'target': 2640, 'unit': 'kcal', 'critical': True}
}

In [5]:
# ======================
# 3. DATA PROCESSING PIPELINE
# ======================

def prepare_postpartum_dataset(food, nutrient, food_nutrient, category):
    """Create analysis-ready dataset with safety checks"""
    # Merge datasets
    df = (
        food_nutrient.merge(food, on='fdc_id', how='inner', validate='many_to_one')
                    .merge(nutrient, left_on='nutrient_id', right_on='id', how='left', validate='many_to_one')
                    .merge(category, left_on='food_category_id', right_on='id', how='left', validate='many_to_one')
    )
    
    # Validate merge
    assert len(df) > 100000, "Merge resulted in insufficient records"
    
    # Pivot to wide format (food × nutrients)
    pivot_df = df.pivot_table(
        index=['fdc_id', 'description', 'description_y'],  # Food ID, name, category
        columns='name',
        values='amount',
        aggfunc='mean'  # Handle duplicates by averaging
    ).reset_index()
    
    # Fill NA safely
    for nut in POSTPARTUM_RDA:
        if nut in pivot_df.columns:
            pivot_df[nut] = pivot_df[nut].fillna(0)
    
    # Create labels (meets ≥15% of any critical nutrient)
    critical_nutrients = [k for k,v in POSTPARTUM_RDA.items() if v['critical']]
    for nut in critical_nutrients:
        if nut in pivot_df.columns:
            pivot_df[f'{nut}_label'] = (
                pivot_df[nut] >= 0.15 * POSTPARTUM_RDA[nut]['target']
            ).astype(int)
    
    pivot_df['postpartum_supportive'] = pivot_df[[f'{n}_label' for n in critical_nutrients]].max(axis=1)
    
    return pivot_df

In [6]:
# ======================
# 4. MODEL TRAINING WITH VALIDATION
# ======================

def train_postpartum_model(data):
    """Train and validate Random Forest classifier"""
    try:
        # Select features and target
        features = [k for k in POSTPARTUM_RDA.keys() if k in data.columns]
        X = data[features]
        y = data['postpartum_supportive']
        
        # Train-test split with stratification
        X_train, X_test, y_train, y_test = train_test_split(
            X, y, test_size=0.2, random_state=42, stratify=y)
        
        # Scale features
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        # Train Random Forest (best for this use case)
        model = RandomForestClassifier(
            n_estimators=100,
            max_depth=10,
            min_samples_leaf=5,
            class_weight='balanced',  # Handle class imbalance
            random_state=42
        )
        model.fit(X_train_scaled, y_train)
        
        # Evaluate
        y_pred = model.predict(X_test_scaled)
        print("Model Evaluation:")
        print(f"Accuracy: {accuracy_score(y_test, y_pred):.2f}")
        print("\nConfusion Matrix:")
        print(confusion_matrix(y_test, y_pred))
        print("\nClassification Report:")
        print(classification_report(y_test, y_pred))
        
        # Feature importance
        plt.figure(figsize=(10,6))
        feat_importances = pd.Series(model.feature_importances_, index=features)
        feat_importances.nlargest(10).plot(kind='barh')
        plt.title("Top 10 Important Nutrients for Postpartum Health")
        plt.show()
        
        return model, scaler, features
    
    except Exception as e:
        print(f"Model training failed: {str(e)}")
        raise


In [7]:
# ======================
# 5. SAFE RECOMMENDATION ENGINE
# ======================

class PostpartumNutritionAdvisor:
    def __init__(self, model, scaler, features, data):
        self.model = model
        self.scaler = scaler
        self.features = features
        self.data = data
    
    def get_recommendations(self, user_profile, top_n=5):
        """Generate safe, personalized recommendations"""
        try:
            # Apply filters
            filtered = self._apply_filters(user_profile)
            
            if len(filtered) == 0:
                return pd.DataFrame(), "No foods match your criteria"
            
            # Prepare features
            X = filtered[self.features]
            X_scaled = self.scaler.transform(X)
            
            # Predict
            filtered['prediction'] = self.model.predict(X_scaled)
            recommendations = filtered[filtered['prediction'] == 1]
            
            if len(recommendations) == 0:
                return pd.DataFrame(), "No suitable recommendations found"
            
            # Prioritize by nutrient density
            recommendations['score'] = recommendations.apply(
                lambda row: sum(
                    row[n] / POSTPARTUM_RDA[n]['target'] * (1 if POSTPARTUM_RDA[n]['critical'] else 0.5)
                    for n in self.features if n in row
                ), axis=1)
            
            # Safety check - ensure minimum nutrient thresholds
            valid_recs = recommendations[
                (recommendations[self.features] > 0).any(axis=1)
            ].sort_values('score', ascending=False).head(top_n)
            
            if len(valid_recs) == 0:
                return pd.DataFrame(), "Recommendations didn't meet safety thresholds"
            
            return valid_recs, "Success"
            
        except Exception as e:
            print(f"Recommendation error: {str(e)}")
            return pd.DataFrame(), "System error"

    def _apply_filters(self, user_profile):
        """Apply filters with safety checks"""
        filtered = self.data.copy()
        
        # Dietary restrictions
        if user_profile.get('vegetarian', False):
            filtered = filtered[~filtered['description_y'].str.contains(
                'Meat|Poultry|Seafood', case=True, na=False)]
        
        if user_profile.get('vegan', False):
            filtered = filtered[~filtered['description_y'].str.contains(
                'Dairy|Egg|Meat', case=True, na=False)]
        
        # Allergies
        if user_profile.get('allergies'):
            allergy_keywords = {
                'dairy': 'Cheese|Milk|Yogurt|Butter',
                'nuts': 'Almond|Walnut|Peanut|Cashew',
                'gluten': 'Wheat|Barley|Rye'
            }
            for allergy in user_profile['allergies']:
                if allergy.lower() in allergy_keywords:
                    filtered = filtered[~filtered['description'].str.contains(
                        allergy_keywords[allergy.lower()], case=False)]
        
        # Focus on deficiencies
        if user_profile.get('deficiency'):
            nut = user_profile['deficiency']
            if nut in self.features:
                filtered = filtered[filtered[nut] >= 0.1 * POSTPARTUM_RDA[nut]['target']]
        
        return filtered


In [8]:
def load_data_safely():
    """Load all USDA datasets with validation checks"""
    try:
        # Load all 5 datasets with essential columns
        food = pd.read_csv('food.csv', usecols=['fdc_id', 'description', 'food_category_id'])
        nutrient = pd.read_csv('nutrient.csv', usecols=['id', 'name', 'unit_name'])
        food_nutrient = pd.read_csv('food_nutrient.csv', 
                                  usecols=['fdc_id', 'nutrient_id', 'amount'])
        category = pd.read_csv('food_category.csv', usecols=['id', 'description'])
        
        # Try loading measure_unit with flexible columns
        try:
            measure_unit = pd.read_csv('measure_unit.csv', 
                                     usecols=['id', 'name', 'abbreviation'])
        except ValueError:
            # Fallback if abbreviation column doesn't exist
            measure_unit = pd.read_csv('measure_unit.csv', 
                                     usecols=['id', 'name'])
        
        # Validate critical columns
        required_columns = {
            'food': ['fdc_id'],
            'nutrient': ['id'],
            'food_nutrient': ['amount'],
            'category': ['id'],
            'measure_unit': ['id']
        }
        
        for df_name, df in zip(['food', 'nutrient', 'food_nutrient', 'category', 'measure_unit'],
                             [food, nutrient, food_nutrient, category, measure_unit]):
            missing = set(required_columns[df_name]) - set(df.columns)
            if missing:
                raise ValueError(f"Missing columns in {df_name}: {missing}")
        
        return food, nutrient, food_nutrient, category, measure_unit
    
    except Exception as e:
        print(f"🚨 Data loading failed: {str(e)}")
        raise

In [9]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split

# Enhanced RDA for Postpartum Mothers
POSTPARTUM_RDA = {
    'Energy': {'target': 2640, 'weight': 0.1, 'unit': 'kcal'},
    'Protein': {'target': 71, 'weight': 0.15, 'unit': 'g'},
    'Vitamin A': {'target': 1300, 'weight': 0.05, 'unit': 'μg RE'},
    'Vitamin D': {'target': 5, 'weight': 0.05, 'unit': 'μg'},
    'Vitamin E': {'target': 19, 'weight': 0.05, 'unit': 'mg'},
    'Folic Acid': {'target': 600, 'weight': 0.1, 'unit': 'μg'},
    'Thiamin': {'target': 1.4, 'weight': 0.03, 'unit': 'mg'},
    'Riboflavin': {'target': 1.6, 'weight': 0.03, 'unit': 'mg'},
    'Niacin': {'target': 17, 'weight': 0.04, 'unit': 'mg'},
    'Vitamin B12': {'target': 2.8, 'weight': 0.05, 'unit': 'μg'},
    'Vitamin B6': {'target': 2.0, 'weight': 0.03, 'unit': 'mg'},
    'Vitamin C': {'target': 120, 'weight': 0.05, 'unit': 'mg'},
    'Iron, Fe': {'target': 9, 'weight': 0.05, 'unit': 'mg'},
    'Zinc': {'target': 12, 'weight': 0.04, 'unit': 'mg'},
    'Iodine': {'target': 290, 'weight': 0.03, 'unit': 'μg'},
    'Copper': {'target': 1300, 'weight': 0.02, 'unit': 'μg'},
    'Selenium': {'target': 70, 'weight': 0.02, 'unit': 'μg'}
}


# Load data from USDA + Homemade recipes
def load_enhanced_data():
    try:
        usda_food = pd.read_csv('food.csv')[['fdc_id', 'description', 'food_category_id']]
        nutrient = pd.read_csv('nutrient.csv')[['id', 'name', 'unit_name']]
        food_nutrient = pd.read_csv('food_nutrient.csv')[['fdc_id', 'nutrient_id', 'amount']]

        df = (food_nutrient.merge(usda_food, on='fdc_id')
                          .merge(nutrient, left_on='nutrient_id', right_on='id'))

        df = df.pivot_table(index='description', columns='name', values='amount', aggfunc='sum').reset_index()
    except:
        df = pd.DataFrame()  # Handle case where files aren't present

    # Homemade items with approximated nutritional values (per serving)
    homemade_foods = [
        {'description': 'Lentil Soup', 'Protein': 18, 'Iron, Fe': 3.3, 'Calcium, Ca': 19, 'Folate, total': 181, 'Energy': 230, 'Vitamin B-12': 0},
        {'description': 'Spinach Dal', 'Protein': 12, 'Iron, Fe': 2.7, 'Calcium, Ca': 99, 'Folate, total': 194, 'Energy': 180, 'Vitamin B-12': 0},
        {'description': 'Ragi Porridge', 'Protein': 6, 'Iron, Fe': 4.5, 'Calcium, Ca': 344, 'Folate, total': 30, 'Energy': 200, 'Vitamin B-12': 0},
        {'description': 'Methi Paratha with Curd', 'Protein': 10, 'Iron, Fe': 2.2, 'Calcium, Ca': 120, 'Folate, total': 70, 'Energy': 320, 'Vitamin B-12': 0.8},
        {'description': 'Chicken Stew with Veggies', 'Protein': 28, 'Iron, Fe': 2.5, 'Calcium, Ca': 60, 'Folate, total': 45, 'Energy': 340, 'Vitamin B-12': 1.2},
        {'description': 'Oats and Banana Smoothie', 'Protein': 8, 'Iron, Fe': 1.6, 'Calcium, Ca': 250, 'Folate, total': 20, 'Energy': 210, 'Vitamin B-12': 0.5},
        {'description': 'Idli with Sambar', 'Protein': 9, 'Iron, Fe': 2.5, 'Calcium, Ca': 80, 'Folate, total': 60, 'Energy': 280, 'Vitamin B-12': 0},
        {'description': 'Vegetable Upma', 'Protein': 6, 'Iron, Fe': 1.8, 'Calcium, Ca': 40, 'Folate, total': 55, 'Energy': 250, 'Vitamin B-12': 0},
        {'description': 'Avial (Mixed Veg Curry with Coconut)', 'Protein': 4, 'Iron, Fe': 1.4, 'Calcium, Ca': 90, 'Folate, total': 40, 'Energy': 180, 'Vitamin B-12': 0},
        {'description': 'Coconut Rice with Curd', 'Protein': 7, 'Iron, Fe': 1.5, 'Calcium, Ca': 110, 'Folate, total': 38, 'Energy': 320, 'Vitamin B-12': 0.4},
        {'description': 'Vegetable Kurma with Parotta', 'Protein': 9, 'Iron, Fe': 2.3, 'Calcium, Ca': 70, 'Folate, total': 65, 'Energy': 390, 'Vitamin B-12': 0.2},
        {'description': 'Mutton Curry with Rice', 'Protein': 26, 'Iron, Fe': 3.1, 'Calcium, Ca': 55, 'Folate, total': 32, 'Energy': 450, 'Vitamin B-12': 2.4},
        {'description': 'Fish Curry with Brown Rice', 'Protein': 22, 'Iron, Fe': 2.8, 'Calcium, Ca': 85, 'Folate, total': 35, 'Energy': 370, 'Vitamin B-12': 2.1},
        {'description': 'Egg Curry with Chapati', 'Protein': 18, 'Iron, Fe': 2.2, 'Calcium, Ca': 50, 'Folate, total': 25, 'Energy': 310, 'Vitamin B-12': 1.0},
        {'description': 'Chicken Biryani (Homestyle)', 'Protein': 24, 'Iron, Fe': 2.6, 'Calcium, Ca': 70, 'Folate, total': 30, 'Energy': 450, 'Vitamin B-12': 1.5},
        {'description': 'Egg Dosa with Veg Filling', 'Protein': 14, 'Iron, Fe': 2.0, 'Calcium, Ca': 60, 'Folate, total': 50, 'Energy': 340, 'Vitamin B-12': 0.9},
        {'description': 'Chicken Keema Idli', 'Protein': 20, 'Iron, Fe': 2.3, 'Calcium, Ca': 90, 'Folate, total': 36, 'Energy': 400, 'Vitamin B-12': 1.6}
    ]

    homemade_df = pd.DataFrame(homemade_foods)
    df = pd.concat([df, homemade_df], ignore_index=True).fillna(0)

    return df


# User Input
def get_user_profile():
    print("\n=== Postpartum Nutrition Profile ===")
    return {
        'breastfeeding': input("Are you breastfeeding? (yes/no): ").lower() == 'yes',
        'diet_type': input("Diet type (vegetarian/vegan/omnivore): ").lower(),
        'allergies': input("Allergies (comma separated): ").lower().split(','),
        'deficiency': input("Any known deficiency? (iron/b12/none): ").lower(),
        'preferred_cuisine': input("Preferred cuisine (indian/mediterranean/etc): ").lower()
    }

# Recommender Class
class PostpartumRecommender:
    def __init__(self):
        self.data = load_enhanced_data()
        self.nutrients = list(POSTPARTUM_RDA.keys())
        self._prepare_data()
        self._train_model()
    
    def _prepare_data(self):
        for nut in self.nutrients:
            if nut not in self.data.columns:
                self.data[nut] = 0
        
        self.data['postpartum_score'] = self.data.apply(
            lambda row: sum(
                row[n] / POSTPARTUM_RDA[n]['target'] * POSTPARTUM_RDA[n]['weight']
                for n in self.nutrients
            ), axis=1)
        
        threshold = self.data['postpartum_score'].quantile(0.7)
        self.data['recommended'] = (self.data['postpartum_score'] >= threshold).astype(int)
    
    def _train_model(self):
        X = self.data[self.nutrients]
        y = self.data['recommended']
        self.scaler = MinMaxScaler()
        X_scaled = self.scaler.fit_transform(X)
        self.model = RandomForestClassifier(class_weight='balanced')
        self.model.fit(X_scaled, y)

    def recommend(self, user_profile):
        filtered = self.data.copy()

        # Diet filter
        if user_profile['diet_type'] == 'vegetarian':
            filtered = filtered[~filtered['description'].str.contains('meat|chicken|fish|egg', case=False, na=False)]
        elif user_profile['diet_type'] == 'vegan':
            filtered = filtered[~filtered['description'].str.contains('meat|chicken|fish|egg|milk|curd|paneer|cheese', case=False, na=False)]

        # Allergy filter
        for allergy in user_profile['allergies']:
            allergy = allergy.strip()
            if allergy:
                filtered = filtered[~filtered['description'].str.contains(allergy, case=False, na=False)]

        # Deficiency preference
        if user_profile['deficiency'] == 'iron' and 'Iron, Fe' in filtered.columns:
            filtered = filtered.sort_values('Iron, Fe', ascending=False)
        elif user_profile['deficiency'] == 'b12' and 'Vitamin B-12' in filtered.columns:
            filtered = filtered.sort_values('Vitamin B-12', ascending=False)

        # Predict
        X = filtered[self.nutrients]
        X_scaled = self.scaler.transform(X)
        filtered['prediction'] = self.model.predict(X_scaled)
        result = filtered[filtered['prediction'] == 1].sort_values('postpartum_score', ascending=False).head(10)

        return result

# Main Execution
if __name__ == "__main__":
    print("=== Postpartum Nutrition Advisor ===")
    user_profile = get_user_profile()
    advisor = PostpartumRecommender()
    recommendations = advisor.recommend(user_profile)

    print("\n🔬 Personalized Recommendations:")
    for idx, row in recommendations.iterrows():
        print(f"\n- {row['description']}")
        print("  Nutrients per serving:")
        for nut in advisor.nutrients:
            print(f"  {nut}: {row[nut]}{POSTPARTUM_RDA[nut]['unit']} ({row[nut]/POSTPARTUM_RDA[nut]['target']*100:.1f}% RDA)")

    print("\n📌 Note: Always consult a healthcare provider for personalized advice.")


=== Postpartum Nutrition Advisor ===

=== Postpartum Nutrition Profile ===

🔬 Personalized Recommendations:

- Mutton Curry with Rice
  Nutrients per serving:
  Energy: 450kcal (17.0% RDA)
  Protein: 26g (36.6% RDA)
  Vitamin A: 0μg RE (0.0% RDA)
  Vitamin D: 0μg (0.0% RDA)
  Vitamin E: 0mg (0.0% RDA)
  Folic Acid: 0μg (0.0% RDA)
  Thiamin: 0mg (0.0% RDA)
  Riboflavin: 0mg (0.0% RDA)
  Niacin: 0mg (0.0% RDA)
  Vitamin B12: 0μg (0.0% RDA)
  Vitamin B6: 0mg (0.0% RDA)
  Vitamin C: 0mg (0.0% RDA)
  Iron, Fe: 3.1mg (34.4% RDA)
  Zinc: 0mg (0.0% RDA)
  Iodine: 0μg (0.0% RDA)
  Copper: 0μg (0.0% RDA)
  Selenium: 0μg (0.0% RDA)

📌 Note: Always consult a healthcare provider for personalized advice.


In [10]:
import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
import re

# Enhanced RDA for Postpartum Mothers
POSTPARTUM_RDA = {
    'Energy': {'target': 2640, 'weight': 0.1, 'unit': 'kcal'},
    'Protein': {'target': 71, 'weight': 0.15, 'unit': 'g'},
    'Vitamin A': {'target': 1300, 'weight': 0.05, 'unit': 'μg RE'},
    'Vitamin D': {'target': 5, 'weight': 0.05, 'unit': 'μg'},
    'Vitamin E': {'target': 19, 'weight': 0.05, 'unit': 'mg'},
    'Folic Acid': {'target': 600, 'weight': 0.1, 'unit': 'μg'},
    'Thiamin': {'target': 1.4, 'weight': 0.03, 'unit': 'mg'},
    'Riboflavin': {'target': 1.6, 'weight': 0.03, 'unit': 'mg'},
    'Niacin': {'target': 17, 'weight': 0.04, 'unit': 'mg'},
    'Vitamin B12': {'target': 2.8, 'weight': 0.05, 'unit': 'μg'},
    'Vitamin B6': {'target': 2.0, 'weight': 0.03, 'unit': 'mg'},
    'Vitamin C': {'target': 120, 'weight': 0.05, 'unit': 'mg'},
    'Iron, Fe': {'target': 9, 'weight': 0.05, 'unit': 'mg'},
    'Zinc': {'target': 12, 'weight': 0.04, 'unit': 'mg'},
    'Iodine': {'target': 290, 'weight': 0.03, 'unit': 'μg'},
    'Copper': {'target': 1300, 'weight': 0.02, 'unit': 'μg'},
    'Selenium': {'target': 70, 'weight': 0.02, 'unit': 'μg'}
}


# Load and clean food data
def load_enhanced_data():
    try:
        usda_food = pd.read_csv('food.csv')[['fdc_id', 'description', 'food_category_id']]
        nutrient = pd.read_csv('nutrient.csv')[['id', 'name', 'unit_name']]
        food_nutrient = pd.read_csv('food_nutrient.csv')[['fdc_id', 'nutrient_id', 'amount']]

        df = (food_nutrient.merge(usda_food, on='fdc_id')
                          .merge(nutrient, left_on='nutrient_id', right_on='id'))

        df = df.pivot_table(index='description', columns='name', values='amount', aggfunc='sum').reset_index()

        # Filter: Keep only human-consumable and familiar items
        df = df[~df['description'].str.contains(
            'baby|dog|cat|extract|powder|raw|infant formula|dry|dry mix|imitation|spread|supplement|unknown|unidentified|test',
            case=False, na=False
        )]
    except:
        df = pd.DataFrame()  # fallback if files are missing

    # South Indian homemade items
    homemade_foods = [
        {'description': 'Lentil Soup', 'Protein': 18, 'Iron, Fe': 3.3, 'Folic Acid': 181, 'Energy': 230, 'Vitamin B12': 0},
        {'description': 'Spinach Dal', 'Protein': 12, 'Iron, Fe': 2.7, 'Folic Acid': 194, 'Energy': 180, 'Vitamin B12': 0},
        {'description': 'Ragi Porridge', 'Protein': 6, 'Iron, Fe': 4.5, 'Folic Acid': 30, 'Energy': 200, 'Vitamin B12': 0},
        {'description': 'Methi Paratha with Curd', 'Protein': 10, 'Iron, Fe': 2.2, 'Folic Acid': 70, 'Energy': 320, 'Vitamin B12': 0.8},
        {'description': 'Chicken Stew with Veggies', 'Protein': 28, 'Iron, Fe': 2.5, 'Folic Acid': 45, 'Energy': 340, 'Vitamin B12': 1.2},
        {'description': 'Oats and Banana Smoothie', 'Protein': 8, 'Iron, Fe': 1.6, 'Folic Acid': 20, 'Energy': 210, 'Vitamin B12': 0.5},
        {'description': 'Idli with Sambar', 'Protein': 9, 'Iron, Fe': 2.5, 'Folic Acid': 60, 'Energy': 280, 'Vitamin B12': 0},
        {'description': 'Vegetable Upma', 'Protein': 6, 'Iron, Fe': 1.8, 'Folic Acid': 55, 'Energy': 250, 'Vitamin B12': 0},
        {'description': 'Avial (Mixed Veg Curry with Coconut)', 'Protein': 4, 'Iron, Fe': 1.4, 'Folic Acid': 40, 'Energy': 180, 'Vitamin B12': 0},
        {'description': 'Coconut Rice with Curd', 'Protein': 7, 'Iron, Fe': 1.5, 'Folic Acid': 38, 'Energy': 320, 'Vitamin B12': 0.4},
        {'description': 'Vegetable Kurma with Parotta', 'Protein': 9, 'Iron, Fe': 2.3, 'Folic Acid': 65, 'Energy': 390, 'Vitamin B12': 0.2},
        {'description': 'Mutton Curry with Rice', 'Protein': 26, 'Iron, Fe': 3.1, 'Folic Acid': 32, 'Energy': 450, 'Vitamin B12': 2.4},
        {'description': 'Fish Curry with Brown Rice', 'Protein': 22, 'Iron, Fe': 2.8, 'Folic Acid': 35, 'Energy': 370, 'Vitamin B12': 2.1},
        {'description': 'Egg Curry with Chapati', 'Protein': 18, 'Iron, Fe': 2.2, 'Folic Acid': 25, 'Energy': 310, 'Vitamin B12': 1.0},
        {'description': 'Chicken Biryani (Homestyle)', 'Protein': 24, 'Iron, Fe': 2.6, 'Folic Acid': 30, 'Energy': 450, 'Vitamin B12': 1.5},
        {'description': 'Egg Dosa with Veg Filling', 'Protein': 14, 'Iron, Fe': 2.0, 'Folic Acid': 50, 'Energy': 340, 'Vitamin B12': 0.9},
        {'description': 'Chicken Keema Idli', 'Protein': 20, 'Iron, Fe': 2.3, 'Folic Acid': 36, 'Energy': 400, 'Vitamin B12': 1.6}
    ]

    homemade_df = pd.DataFrame(homemade_foods)
    df = pd.concat([df, homemade_df], ignore_index=True).fillna(0)

    return df


def get_user_profile():
    print("\n=== Postpartum Nutrition Profile ===")
    return {
        'breastfeeding': input("Are you breastfeeding? (yes/no): ").lower() == 'yes',
        'diet_type': input("Diet type (vegetarian/vegan/omnivore): ").lower(),
        'allergies': input("Allergies (comma separated): ").lower().split(','),
        'deficiency': input("Any known deficiency? (iron/b12/none): ").lower(),
        'preferred_cuisine': input("Preferred cuisine (indian/mediterranean/etc): ").lower()
    }


class PostpartumRecommender:
    def __init__(self):
        self.data = load_enhanced_data()
        self.nutrients = list(POSTPARTUM_RDA.keys())
        self._prepare_data()
        self._train_model()

    def _prepare_data(self):
        for nut in self.nutrients:
            if nut not in self.data.columns:
                self.data[nut] = 0

        self.data['postpartum_score'] = self.data.apply(
            lambda row: sum(
                row[n] / POSTPARTUM_RDA[n]['target'] * POSTPARTUM_RDA[n]['weight']
                for n in self.nutrients
            ), axis=1)
        
        threshold = self.data['postpartum_score'].quantile(0.7)
        self.data['recommended'] = (self.data['postpartum_score'] >= threshold).astype(int)

    def _train_model(self):
        X = self.data[self.nutrients]
        y = self.data['recommended']
        self.scaler = MinMaxScaler()
        X_scaled = self.scaler.fit_transform(X)
        self.model = RandomForestClassifier(class_weight='balanced')
        self.model.fit(X_scaled, y)

    def recommend(self, user_profile):
        filtered = self.data.copy()

        # Diet filtering
        if user_profile['diet_type'] == 'vegetarian':
            filtered = filtered[~filtered['description'].str.contains(r'\b(chicken|mutton|fish|beef|egg)\b', case=False, na=False)]
        elif user_profile['diet_type'] == 'vegan':
            filtered = filtered[~filtered['description'].str.contains(r'\b(chicken|mutton|fish|beef|egg|milk|curd|paneer|cheese|butter|ghee)\b', case=False, na=False)]

        # Allergy filtering
        for allergy in user_profile['allergies']:
            allergy = allergy.strip()
            if allergy:
                filtered = filtered[~filtered['description'].str.contains(allergy, case=False, na=False)]

        # Cuisine filtering
        if user_profile['preferred_cuisine'] == 'indian':
            filtered = filtered[filtered['description'].str.contains(r'\b(dal|idli|sambar|biryani|paratha|kurma|upma|poha|keema|chutney|rice|chapati|rasam|poriyal|avial|veg|masala)\b', case=False, na=False)]

        # Deficiency-based boost
        if user_profile['deficiency'] == 'iron' and 'Iron, Fe' in filtered.columns:
            filtered = filtered.sort_values('Iron, Fe', ascending=False)
        elif user_profile['deficiency'] == 'b12' and 'Vitamin B12' in filtered.columns:
            filtered = filtered.sort_values('Vitamin B12', ascending=False)

        # Prediction
        X = filtered[self.nutrients]
        X_scaled = self.scaler.transform(X)
        filtered['prediction'] = self.model.predict(X_scaled)
        result = filtered[filtered['prediction'] == 1].sort_values('postpartum_score', ascending=False).head(10)

        return result


# Execution
if __name__ == "__main__":
    print("=== Postpartum Nutrition Advisor ===")
    user_profile = get_user_profile()
    advisor = PostpartumRecommender()
    recommendations = advisor.recommend(user_profile)

    print("\n🔬 Personalized Recommendations:")
    for idx, row in recommendations.iterrows():
        print(f"\n- {row['description']}")
        print("  Nutrients per serving:")
        for nut in advisor.nutrients:
            if row[nut] > 0:
                unit = POSTPARTUM_RDA[nut]['unit']
                percent = row[nut] / POSTPARTUM_RDA[nut]['target'] * 100
                print(f"    {nut}: {row[nut]:.2f}{unit} ({percent:.1f}% RDA)")

    print("\n📌 Note: Always consult a healthcare provider for personalized advice.")


=== Postpartum Nutrition Advisor ===

=== Postpartum Nutrition Profile ===



🔬 Personalized Recommendations:

- Mutton Curry with Rice
  Nutrients per serving:
    Energy: 450.00kcal (17.0% RDA)
    Protein: 26.00g (36.6% RDA)
    Folic Acid: 32.00μg (5.3% RDA)
    Vitamin B12: 2.40μg (85.7% RDA)
    Iron, Fe: 3.10mg (34.4% RDA)

- Fish Curry with Brown Rice
  Nutrients per serving:
    Energy: 370.00kcal (14.0% RDA)
    Protein: 22.00g (31.0% RDA)
    Folic Acid: 35.00μg (5.8% RDA)
    Vitamin B12: 2.10μg (75.0% RDA)
    Iron, Fe: 2.80mg (31.1% RDA)

- Chicken Biryani (Homestyle)
  Nutrients per serving:
    Energy: 450.00kcal (17.0% RDA)
    Protein: 24.00g (33.8% RDA)
    Folic Acid: 30.00μg (5.0% RDA)
    Vitamin B12: 1.50μg (53.6% RDA)
    Iron, Fe: 2.60mg (28.9% RDA)

- Chicken Keema Idli
  Nutrients per serving:
    Energy: 400.00kcal (15.2% RDA)
    Protein: 20.00g (28.2% RDA)
    Folic Acid: 36.00μg (6.0% RDA)
    Vitamin B12: 1.60μg (57.1% RDA)
    Iron, Fe: 2.30mg (25.6% RDA)

📌 Note: Always consult a healthcare provider for personalized advice.


C:\Users\Varshitha  N S\AppData\Local\Temp\ipykernel_29796\2389291375.py:134: UserWarning: This pattern is interpreted as a regular expression, and has match groups. To actually get the groups, use str.extract.
  filtered = filtered[filtered['description'].str.contains(r'\b(dal|idli|sambar|biryani|paratha|kurma|upma|poha|keema|chutney|rice|chapati|rasam|poriyal|avial|veg|masala)\b', case=False, na=False)]
